# TavilySearchResults

[Tavily](https://tavily.com/) Search is a robust search API tailored specifically for LLM Agents. It seamlessly integrates with diverse data sources to ensure a superior, relevant search experience.

This guide provides a quick overview for getting started with the Tavily search results [tool](/docs/integrations/tools/). For detailed documentation of all `TavilySearchResults` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_tools_tavily_search.TavilySearchResults.html).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/tools/tavily_search/) | Package latest |
| :--- | :--- | :---: | :---: |
| [TavilySearchResults](https://api.js.langchain.com/classes/langchain_community_tools_tavily_search.TavilySearchResults.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## Setup

The integration lives in the `@langchain/community` package, which you can install as shown below:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>
```

### Credentials

Set up an API key [here](https://app.tavily.com) and set it as an environment variable named `TAVILY_API_KEY`.

```typescript
process.env.TAVILY_API_KEY = "YOUR_API_KEY"
```

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGCHAIN_TRACING_V2="true"
process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

You can import and instantiate an instance of the `TavilySearchResults` tool like this:

In [1]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tool = new TavilySearchResults({
  maxResults: 2,
  // ...
});

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

You can invoke the tool directly like this:

In [2]:
await tool.invoke({
  input: "what is the current weather in SF?"
});

[{"title":"San Francisco, CA Current Weather | AccuWeather","url":"https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629","content":"Current weather in San Francisco, CA. Check current conditions in San Francisco, CA with radar, hourly, and more.","score":0.9428234,"raw_content":null},{"title":"National Weather Service","url":"https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA","content":"NOAA National Weather Service. Current conditions at SAN FRANCISCO DOWNTOWN (SFOC1) Lat: 37.77056°NLon: 122.42694°WElev: 150.0ft.","score":0.94261247,"raw_content":null}]


### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "what is the current weather in SF?"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "[{\"title\":\"Weather in San Francisco\",\"url\":\"https://www.weatherapi.com/\",\"content\":\"{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722967498, 'localtime': '2024-08-06 11:04'}, 'current': {'last_updated_epoch': 1722967200, 'last_updated': '2024-08-06 11:00', 'temp_c': 18.4, 'temp_f': 65.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.9, 'wind_kph': 4.7, 'wind_degree': 275, 'wind_dir': 'W', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 2, 'feelslike_c': 18.5, 'feelslike_f': 65.2, 'windchill_c': 18.5, 'windchill_f': 65.2, 'heatindex_c': 18.4, 'heatindex_f': 65.2, 'dewpoint_c': 11.7, 'dewpoint_f': 53.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.0, 'gust_mph': 4.3, 'gust_kph

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [7]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("what is the current weather in sf?");

In [8]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

AIMessage {
  "tool_calls": [],
  "content": "The current weather in San Francisco is as follows:\n\n- **Condition:** Sunny\n- **Temperature:** 18.4°C (65.2°F)\n- **Wind:** 2.9 mph (4.7 kph) from the west\n- **Humidity:** 64%\n- **Visibility:** 10 km (6 miles)\n- **UV Index:** 5\n\n![Sunny](//cdn.weatherapi.com/weather/64x64/day/113.png)\n\nFor more detailed information, you can visit [WeatherAPI](https://www.weatherapi.com/)."
}


## Agents

For guides on how to use LangChain tools in agents, see the [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) docs.

## API reference

For detailed documentation of all `TavilySearchResults` features and configurations head to the API reference: https://api.js.langchain.com/classes/langchain_community_tools_tavily_search.TavilySearchResults.html